# Plot images and their bboxes in annotation files

The bboxes of each image is contained in the annotation file (i.e., one JSON file per image). For the annotation format, see section *Train with the Image Format* in the [SageMaker Object Detection Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html) documentation.

Recall the annotation structure:

```json
{
   "file": "haha.jpg",
   "image_size": [...]
   "annotations": [
      {
         "class_id": 0,
         "left": 300,
         "top": 38,
         "width": 100,
         "height": 52
      },
      {...}
   ],
   "categories": [...]
}
```

We'll loop over all annotation files in S3. For each annotation file, load the corresponding image, then plot image + bboxes.

In [ ]:
import s3fs
import json
import matplotlib.pyplot as plt
import numpy as np

# Prefer opencv, as with PIL there's no way to load single-channel jpg as rgb in-memory data.
#from PIL import Image, ImageDraw
import cv2

json_path = 's3://bucket/dataset/train_annotation/'
img_path = 's3://bucket/dataset/train/'
fs = s3fs.S3FileSystem(anon=False, profile_name='default')

# Get the name of all JSON files.
json_files = fs.ls(json_path)

# Will plot images in two columns
nrows, ncols = (len(json_files)+1)//2, 2
w,h = 16, 18
_, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(w,h))

# Iterate through all annotation json files
for i, json_fname in enumerate(json_files):
    # Deserialize annotation JSON file to dictionary
    d = json.loads(fs.cat(json_fname))

    # Load image from S3 into 3-channel.
    # NOTE: if using bytearray(), then pair with np.asarray().
    img_fname = os.path.join(img_path, d['file'])
    b = bytes(fs.cat(img_fname))
    arr = np.frombuffer(b, dtype=np.uint8)
    img = cv2.imdecode(arr, 1)

    # Draw bboxes on image
    for bbox in d['annotations']:
        # bbox coordinates
        x_min, y_min = bbox['left'], bbox['top']
        x_max, y_max = x_min + bbox['width'], y_min + bbox['height']

        # color to use (will round-robin r->g->b according to class_id)
        cid = bbox['class_id']
        color = [0,0,0]
        color[cid % 3] = 255
        cv2.rectangle(img, pt1=(x_min, y_min), pt2=(x_max, y_max), color=color, thickness=2)

    # Add to plot figure
    row, col = i//2, i%2
    this_ax = ax[row][col]
    this_ax.set_title(f'[img-{i:02}] {img_fname}')
    this_ax.imshow(img)

plt.show();